In [1]:
from pyspark.sql import SparkSession
import numpy as np

In [3]:
spark = SparkSession.builder.appName('embarazos').getOrCreate()

ruta = '../../Data/DatosCovid/220720COVID19MEXICO.csv'
df = spark.read.csv(
    ruta, 
    header = True,
    inferSchema = True,
    mode = 'PERMISSIVE',
    multiLine = False,
    escape = "\""
)

df.show()

25/10/25 11:17:54 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+-----------+------+------+----------+----+-----------+-----------+-------------+-------------+-------------+--------------+----------+--------+--------+----+------------+--------+------------------+--------+--------+----+----+--------+------------+--------+--------------+--------+-------------+----------+---------+----------------+-------------+---------------------+------------------+-------------------+--------+-----------------+-----------+---+
|FECHA_ACTUALIZACION|ID_REGISTRO|ORIGEN|SECTOR|ENTIDAD_UM|SEXO|ENTIDAD_NAC|ENTIDAD_RES|MUNICIPIO_RES|TIPO_PACIENTE|FECHA_INGRESO|FECHA_SINTOMAS| FECHA_DEF|INTUBADO|NEUMONIA|EDAD|NACIONALIDAD|EMBARAZO|HABLA_LENGUA_INDIG|INDIGENA|DIABETES|EPOC|ASMA|INMUSUPR|HIPERTENSION|OTRA_COM|CARDIOVASCULAR|OBESIDAD|RENAL_CRONICA|TABAQUISMO|OTRO_CASO|TOMA_MUESTRA_LAB|RESULTADO_LAB|TOMA_MUESTRA_ANTIGENO|RESULTADO_ANTIGENO|CLASIFICACION_FINAL|MIGRANTE|PAIS_NACIONALIDAD|PAIS_ORIGEN|UCI|
+-------------------+-----------+------+------+-------

In [4]:
df.dtypes

[('FECHA_ACTUALIZACION', 'string'),
 ('ID_REGISTRO', 'string'),
 ('ORIGEN', 'string'),
 ('SECTOR', 'int'),
 ('ENTIDAD_UM', 'int'),
 ('SEXO', 'int'),
 ('ENTIDAD_NAC', 'string'),
 ('ENTIDAD_RES', 'int'),
 ('MUNICIPIO_RES', 'string'),
 ('TIPO_PACIENTE', 'int'),
 ('FECHA_INGRESO', 'string'),
 ('FECHA_SINTOMAS', 'string'),
 ('FECHA_DEF', 'string'),
 ('INTUBADO', 'string'),
 ('NEUMONIA', 'string'),
 ('EDAD', 'string'),
 ('NACIONALIDAD', 'string'),
 ('EMBARAZO', 'string'),
 ('HABLA_LENGUA_INDIG', 'string'),
 ('INDIGENA', 'string'),
 ('DIABETES', 'string'),
 ('EPOC', 'string'),
 ('ASMA', 'string'),
 ('INMUSUPR', 'string'),
 ('HIPERTENSION', 'string'),
 ('OTRA_COM', 'string'),
 ('CARDIOVASCULAR', 'string'),
 ('OBESIDAD', 'string'),
 ('RENAL_CRONICA', 'string'),
 ('TABAQUISMO', 'string'),
 ('OTRO_CASO', 'string'),
 ('TOMA_MUESTRA_LAB', 'string'),
 ('RESULTADO_LAB', 'int'),
 ('TOMA_MUESTRA_ANTIGENO', 'string'),
 ('RESULTADO_ANTIGENO', 'string'),
 ('CLASIFICACION_FINAL', 'string'),
 ('MIGRANTE', '

In [ ]:
# REGISTROS POR PAIS
result = df.groupBy(df['PAIS_NACIONALIDAD']).count().orderBy('count', ascending = False).collect()
for row in result:
    print(f'{row[1]} - {row[0]}')

17186453 - México
51950 - Estados Unidos de América
9242 - Venezuela
7410 - Colombia
5059 - República de Honduras
4412 - Guatemala
3193 - Cuba
2865 - Argentina
2835 - Nicaragua
2825 - España
2750 - El Salvador
2703 - Brasil
2165 - Alemania
2104 - Otro
1903 - Francia
1847 - Haití
1795 - Canadá
1793 - Perú
1782 - China
1768 - Ecuador
1427 - Japón
1253 - Chile
1235 - Italia
946 - Ucrania
780 - Bolivia
623 - India
488 - República de Corea
451 - Gran Bretaña (Reino Unido)
423 - República Dominicana
388 - Rusia
278 - Suiza
251 - República de Costa Rica
247 - Filipinas
235 - Belice
232 - Holanda
227 - República Oriental del Uruguay
209 - Australia
207 - Polonia
183 - República de Panamá
179 - Portugal
173 - Paraguay
146 - SE DESCONOCE
143 - Rumania
141 - Estado Libre Asociado de Puerto Rico
134 - Bélgica
119 - Israel
112 - Sudáfrica
103 - Irlanda
98 - Turquía
95 - Austria
92 - Dinamarca
86 - Nigeria
84 - República Checa y República Eslovaca
75 - Zona Neutral
73 - Egipto
72 - Antigua y Bermuda

# Experimentos con groupBy

Quiero saber:

* Cuántas mujeres hay por cada país
* Cuántas mujeres hay en total en comparación con los hombres
* Proporción de mujeres embarazadas

In [6]:
'''
Cuántas mujeres hay de cada país
'''
n_mujeres = df.filter(df['SEXO'] == 1).count()
print(f'Hay un total de {n_mujeres} mujeres')
df.filter(df['SEXO'] == 1).groupBy('PAIS_NACIONALIDAD').count().orderBy('count', ascending = False).show()

Hay un total de 9291813 mujeres


+--------------------+-------+
|   PAIS_NACIONALIDAD|  count|
+--------------------+-------+
|              México|9234466|
|Estados Unidos de...|  25705|
|           Venezuela|   5036|
|            Colombia|   4019|
|República de Hond...|   1989|
|           Guatemala|   1804|
|                Cuba|   1433|
|              Brasil|   1252|
|         El Salvador|   1248|
|           Argentina|   1243|
|           Nicaragua|   1036|
|              España|   1034|
|                Otro|    875|
|                Perú|    866|
|             Ecuador|    819|
|               Haití|    807|
|              Canadá|    798|
|             Francia|    790|
|            Alemania|    691|
|               Chile|    558|
+--------------------+-------+
only showing top 20 rows


In [7]:
'''
Cuántas mujeres hay en total en comparación con los hombres
'''
total = df.count()
n_mujeres = df.filter(df['SEXO'] == 1).count()
n_hombres = df.filter(df['SEXO'] == 2).count()

prop_m = n_mujeres / total
prop_h = n_hombres / total

print(f'La proporción de mujeres: {np.round(prop_m * 100, 2)} %')
print(f'La proporción de hombres: {np.round(prop_h * 100, 2)} %')

La proporción de mujeres: 53.68 %
La proporción de hombres: 46.32 %


In [10]:
'''
Proporción embarazadas
'''
total_m = df.filter(df['SEXO'] == 1).count()
si_emb = df.filter((df['SEXO'] == 1) & (df['EMBARAZO'] == 1)).count()
no_emb = df.filter((df['SEXO'] == 1) & (df['EMBARAZO'] == 2)).count()

prop_emb = si_emb / total_m
prop_no_emb = no_emb / total_m

print(f'La proporción de mujeres embarazadas: {np.round(prop_emb * 100, 2)} %')
print(f'La proporción de mujeres no embarazadas: {np.round(prop_no_emb * 100, 2)} %')

La proporción de mujeres embarazadas: 1.92 %
La proporción de mujeres no embarazadas: 97.29 %


### Formatear datos para trabajarlos

In [15]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

numericas = [col for col in df.columns if col != 'PAIS_NACIONALIDAD']
for c in numericas:
    df = df.withColumn(c, col(c).cast(IntegerType()))

df.dtypes

[('FECHA_ACTUALIZACION', 'int'),
 ('ID_REGISTRO', 'int'),
 ('ORIGEN', 'int'),
 ('SECTOR', 'int'),
 ('ENTIDAD_UM', 'int'),
 ('SEXO', 'int'),
 ('ENTIDAD_NAC', 'int'),
 ('ENTIDAD_RES', 'int'),
 ('MUNICIPIO_RES', 'int'),
 ('TIPO_PACIENTE', 'int'),
 ('FECHA_INGRESO', 'int'),
 ('FECHA_SINTOMAS', 'int'),
 ('FECHA_DEF', 'int'),
 ('INTUBADO', 'int'),
 ('NEUMONIA', 'int'),
 ('EDAD', 'int'),
 ('NACIONALIDAD', 'int'),
 ('EMBARAZO', 'int'),
 ('HABLA_LENGUA_INDIG', 'int'),
 ('INDIGENA', 'int'),
 ('DIABETES', 'int'),
 ('EPOC', 'int'),
 ('ASMA', 'int'),
 ('INMUSUPR', 'int'),
 ('HIPERTENSION', 'int'),
 ('OTRA_COM', 'int'),
 ('CARDIOVASCULAR', 'int'),
 ('OBESIDAD', 'int'),
 ('RENAL_CRONICA', 'int'),
 ('TABAQUISMO', 'int'),
 ('OTRO_CASO', 'int'),
 ('TOMA_MUESTRA_LAB', 'int'),
 ('RESULTADO_LAB', 'int'),
 ('TOMA_MUESTRA_ANTIGENO', 'int'),
 ('RESULTADO_ANTIGENO', 'int'),
 ('CLASIFICACION_FINAL', 'int'),
 ('MIGRANTE', 'int'),
 ('PAIS_NACIONALIDAD', 'string'),
 ('PAIS_ORIGEN', 'int'),
 ('UCI', 'int')]

In [16]:
'''
Resultado de lab en mujeres embarazadas
'''
total_emb = df.filter((df['SEXO'] == 1) & (df['EMBARAZO'] == 1))
emb_positivo = df.filter((df['SEXO'] == 1) & (df['EMBARAZO'] == 1) & (df['RESULTADO_LAB'] == 1)).count()

prop_emb_pos = emb_positivo / total_emb

print(f'Resultados positivos de lab de las embarazadas: {np.round(prop_emb_pos * 100, 2)} %')

TypeError: unsupported operand type(s) for /: 'int' and 'DataFrame'